# Ingest Cookbook (QueryGenerator).

Following the concepts on the different [means](https://docs.aperturedata.io/HowToGuides/Ingestion/Ingestion#custom-defined-data-generators) of ingesting the data, we will build an example using the QueryGenerator method in this notebook.

We will use the [Cookbook dataset](https://docs.google.com/spreadsheets/d/1G1HPG3Dxx5W39OD6b74wMHvWupD7N-DLUbV7tD5owx8/edit?usp=sharing) to have the data persisted onto ApertureDB instance.

We merge the first 3 sheets on the source into a json file such that there will be a list of Dishes objects, and each Dish may have multiple ingredients, along with some oher miscellaneous properties.

:::caution

The guide assumes that the SDK has been setup as per the instructions.

Colab one time setup [instructions](/Setup/client/notebooks?notebooks=colab).

Non ephemeral jupyter server one time setup [instructions](/Setup/client/notebooks?notebooks=notebook).

:::


## Needed resources.
- ```create_nested_json.py```: The script merges the first 3 sheets on the source into a json file such that there will be a list of Dishes objects, and each Dish may have multiple ingredients, and each ingredient has miscellaneous properties. This ends up as a json file.


In [1]:
# Get the script to generate the data.json
!wget https://github.com/aperture-data/Cookbook/raw/refs/heads/main/scripts/create_nested_json.py

# Run the script to generate the data.json
!python create_nested_json.py

--2024-10-01 16:55:05--  https://github.com/aperture-data/Cookbook/raw/refs/heads/main/scripts/create_nested_json.py
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aperture-data/Cookbook/refs/heads/main/scripts/create_nested_json.py [following]
--2024-10-01 16:55:05--  https://raw.githubusercontent.com/aperture-data/Cookbook/refs/heads/main/scripts/create_nested_json.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1757 (1.7K) [text/plain]
Saving to: ‘create_nested_json.py.1’

create_nested_json. 100%[===================>]   1.72K  --.-KB/s    in 0s      

2024-10-01 16:55:06 (5.41 MB/s) - ‘c

# Ingest using a Query Generator

When the source data is in a format that does not conform to any of the CSV pasrsers in the SDK, we could use the approach of defining a custom Query generator.

This does require a level of familiarity with the Query language.

Let's implement a class to deal with the cookbook example.

The Query generator is used to define a ```getitem``` to return a query to issue to ApertureDB that persists the record being iterated at the source.

In [3]:
from typing import Tuple
from aperturedb.CommonLibrary import create_connector, execute_query
from aperturedb.QueryGenerator import QueryGenerator
from aperturedb.types import *
from aperturedb.Sources import Sources
import json

from tqdm.auto import tqdm

class CookBookQueryGenerator(QueryGenerator):
    def __init__(self, *args, **kwargs):
        super().__init__()
        assert "dishes" in kwargs, "Path to Dishes must be provided"
        with open(kwargs["dishes"]) as ins:
            self.dishes = self.dishes = json.load(ins)
            print(f"Loaded {len(self.dishes)} dishes")

    def __len__(self) -> int:
        return len(self.dishes)

    def getitem(self, idx: int) -> Tuple[Commands, Blobs]:
        record = self.dishes[idx]
        q = [
            {
                "AddImage":{
                    "_ref": 1,
                    "properties": {
                        "contributor": record["contributor"],
                        "name": record["name"],
                        "location": record["location"],
                        "cuisine": record["cuisine"],
                        "caption": record["caption"],
                        "recipe_url": record["recipe_url"],
                        "dish_id": record["dish_id"]
                    }
                }
            }
        ]
        for i, ingredient in enumerate(record["ingredients"]):
            q.append({
                "AddEntity": {
                    "_ref": 2 + i,
                    "class": "Ingredient",
                    "connect": {
                        "ref": 1
                    },
                    "properties": {
                        "Name": ingredient["Name"],
                        "other_names": ingredient.get("other_names", ""),
                        "macronutrient": ingredient.get("macronutrient", ""),
                        "micronutrient": ingredient.get("micronutrient", ""),
                        "subgroup": ingredient.get("subgroup", ""),
                        "category": ingredient.get("category", "")
                    }
                }
            })

        blob = Sources(n_download_retries=3).load_from_http_url(record["url"], validator=lambda x: True)
        return q, [blob[1]]



client = create_connector()
generator = CookBookQueryGenerator(dishes="dishes.json")
for query, blobs in tqdm(generator):
    result, response, output_blobs = execute_query(client, query, blobs)

    if result != 0:
        print(response, query)
        break

Loaded 29 dishes


  0%|          | 0/29 [00:00<?, ?it/s]

## CSV Parser files are also supported by adb CLI.

The ```CookBookQueryGenerator``` class can be saved in a python file called CookBookQueryGenerator.py
```bash
adb ingest from-generator CookBookQueryGenerator.py
```

:::caution

The filename needs to match the class name for the adb command to work.

:::
